This notebook contains a minimal example of using `fev` to evaluate time series forecasting models.

In [1]:
import fev

In [2]:
# Create a task from a dataset stored on Hugging Face Hub
task = fev.Task(
    dataset_path="autogluon/chronos_datasets",
    dataset_config="monash_kdd_cup_2018",
    horizon=12,
)

In [3]:
# Load data available as input to the forecasting model
past_data, future_data = task.get_input_data()

Validating dataset format (num_proc=48):   0%|          | 0/270 [00:00<?, ? examples/s]

Filtering short time series (num_proc=48):   0%|          | 0/270 [00:00<?, ? examples/s]

Selecting past data (num_proc=48):   0%|          | 0/270 [00:00<?, ? examples/s]

Selecting future data (num_proc=48):   0%|          | 0/270 [00:00<?, ? examples/s]

In [4]:
# past data before the forecast horizon.
past_data

Dataset({
    features: ['id', 'timestamp', 'target', 'city', 'station', 'measurement'],
    num_rows: 270
})

In [5]:
past_data[0]

{'id': 'T000000',
 'timestamp': array(['2017-01-01T14:00:00.000', '2017-01-01T15:00:00.000',
        '2017-01-01T16:00:00.000', ..., '2018-03-31T01:00:00.000',
        '2018-03-31T02:00:00.000', '2018-03-31T03:00:00.000'],
       dtype='datetime64[ms]'),
 'target': array([453., 417., 395., ...,  99., 102.,  97.], dtype=float32),
 'city': 'Beijing',
 'station': 'aotizhongxin_aq',
 'measurement': 'PM2.5'}

In [6]:
# future data that is known at prediction time (item ID, future timestamps, and known covariates)
future_data

Dataset({
    features: ['id', 'timestamp', 'city', 'station', 'measurement'],
    num_rows: 270
})

In [7]:
future_data[0]

{'id': 'T000000',
 'timestamp': array(['2018-03-31T04:00:00.000', '2018-03-31T05:00:00.000',
        '2018-03-31T06:00:00.000', '2018-03-31T07:00:00.000',
        '2018-03-31T08:00:00.000', '2018-03-31T09:00:00.000',
        '2018-03-31T10:00:00.000', '2018-03-31T11:00:00.000',
        '2018-03-31T12:00:00.000', '2018-03-31T13:00:00.000',
        '2018-03-31T14:00:00.000', '2018-03-31T15:00:00.000'],
       dtype='datetime64[ms]'),
 'city': 'Beijing',
 'station': 'aotizhongxin_aq',
 'measurement': 'PM2.5'}

In [8]:
import numpy as np


# Make predictions
def naive_forecast(y: list, horizon: int) -> list:
    return [y[np.isfinite(y)][-1] for _ in range(horizon)]

predictions = []
for ts in past_data:
    predictions.append(
        {"predictions": naive_forecast(y=ts[task.target_column], horizon=task.horizon)}
    )

In [9]:
eval_summary = task.evaluation_summary(predictions, model_name="naive")
eval_summary

Casting the dataset:   0%|          | 0/270 [00:00<?, ? examples/s]

Map (num_proc=48):   0%|          | 0/270 [00:00<?, ? examples/s]

{'model_name': 'naive',
 'dataset_name': 'chronos_datasets_monash_kdd_cup_2018',
 'dataset_path': 'autogluon/chronos_datasets',
 'dataset_config': 'monash_kdd_cup_2018',
 'horizon': 12,
 'cutoff': -12,
 'lead_time': 1,
 'min_context_length': 1,
 'max_context_length': None,
 'seasonality': 1,
 'eval_metric': 'MASE',
 'extra_metrics': [],
 'quantile_levels': None,
 'id_column': 'id',
 'timestamp_column': 'timestamp',
 'target_column': 'target',
 'generate_univariate_targets_from': None,
 'past_dynamic_columns': [],
 'excluded_columns': [],
 'test_error': 3.3784518866750513,
 'training_time_s': None,
 'inference_time_s': None,
 'dataset_fingerprint': 'a22d13d4c1e8641c',
 'trained_on_this_dataset': False,
 'fev_version': '0.5.0',
 'MASE': 3.3784518866750513}

Evaluation summaries produced by different models on different tasks can be aggregated into a single table.

In [10]:
import pandas as pd

summaries = pd.read_csv("https://raw.githubusercontent.com/autogluon/fev/refs/heads/main/benchmarks/example/results/results.csv")
summaries.head()

model_name                                dataset_name  \
0  auto_theta           chronos_datasets_monash_m1_yearly   
1  auto_theta  chronos_datasets_monash_electricity_weekly   
2  auto_theta  chronos_datasets_monash_electricity_weekly   
3  auto_arima           chronos_datasets_monash_m1_yearly   
4  auto_arima  chronos_datasets_monash_electricity_weekly   

                 dataset_path             dataset_config  horizon  \
0  autogluon/chronos_datasets           monash_m1_yearly        8   
1  autogluon/chronos_datasets  monash_electricity_weekly        8   
2  autogluon/chronos_datasets  monash_electricity_weekly        8   
3  autogluon/chronos_datasets           monash_m1_yearly        8   
4  autogluon/chronos_datasets  monash_electricity_weekly        8   

                cutoff  lead_time  min_ts_length  max_context_length  \
0                   -8          1              9                 NaN   
1  2013-01-01T00:00:00          1              9                 NaN   
2  2014-01-01T00:00:00          1              9                 NaN   
3                   -8          1              9                 NaN   
4  2013-01-01T00:00:00          1              9                 NaN   

   seasonality  ... multiple_target_columns past_dynamic_columns  \
0            1  ...                     NaN                   []   
1            1  ...                     NaN                   []   
2            1  ...                     NaN                   []   
3            1  ...                     NaN                   []   
4            1  ...                     NaN                   []   

   excluded_columns test_error training_time_s inference_time_s  \
0                []   4.241262             NaN         7.116432   
1                []   1.428428             NaN         2.812927   
2                []   1.610647             NaN         6.573564   
3                []   3.993800             NaN         8.246975   
4                []   1.720373             NaN        23.514658   

   dataset_fingerprint trained_on_this_dataset fev_version      MASE  
0     296cc3ca5975e847                   False       0.2.1  4.241262  
1     1bf59473dbf463a3                   False       0.2.1  1.428428  
2     1bf59473dbf463a3                   False       0.2.1  1.610647  
3     296cc3ca5975e847                   False       0.2.1  3.993800  
4     1bf59473dbf463a3                   False       0.2.1  1.720373  

[5 rows x 26 columns]

In [11]:
# Evaluation summaries can be provided as dataframes, dicts, JSON or CSV files
fev.leaderboard(summaries)

gmean_relative_error  avg_rank  avg_inference_time_s  \
model_name                                                             
auto_theta                  0.874455  2.000000              5.500974   
auto_arima                  0.887336  2.000000             21.798952   
auto_ets                    0.951193  2.666667              0.736867   
seasonal_naive              1.000000  3.333333              0.003800   

                median_inference_time_s  avg_training_time_s  \
model_name                                                     
auto_theta                     6.573564                  NaN   
auto_arima                    23.514658                  NaN   
auto_ets                       0.741776                  NaN   
seasonal_naive                 0.004139                  NaN   

                median_training_time_s  training_corpus_overlap  num_failures  
model_name                                                                     
auto_theta                         NaN                      0.0             0  
auto_arima                         NaN                      0.0             0  
auto_ets                           NaN                      0.0             0  
seasonal_naive                     NaN                      0.0             0

The `leaderboard` method not only summarizes the results into a single table, but also ensures that all task definitions & dataset fingerprints match across different models. This ensures that the scores are comparable and the comparison is fair.